In [ ]:
!pip install accelerate bitsandbytes peft datasets trl wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
# wandb.login()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("medical_questions_pairs")

In [ ]:
len(dataset["train"])

In [ ]:
c = 0
for i in range(len(dataset["train"])):
#   print(dataset["train"][i]["label"])
  c += dataset["train"][i]["label"]

c, len(dataset["train"])

In [ ]:
train_dataset = dataset["train"].select(range(100))

test_dataset = dataset["train"].select(range(100, 150))

train_dataset, test_dataset


In [ ]:
def create_prompt(sample):

    prompt = '''
    Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Given 2 medical questions Q1 and Q2, predict if both of them are similar or not.
    If they are similar, output only 1 and nothing else. If not, then output only 0 and nothing else.

    Q1 :- {Q1}
    Q2 :- {Q2}
    '''.format(Q1=sample["question_1"], Q2=sample["question_2"])

    prompt = "<s>" + prompt + "</s>"


    return prompt

In [ ]:
import bitsandbytes
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

In [ ]:
tokenizer.bos_token, tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False,
    max_length = 500
)


In [ ]:
prmpt = create_prompt(train_dataset[0])

print(prmpt)

In [ ]:
# prmpt_enc = tokenizer.encode(prmpt)
# model.generate(torch.tensor(prmpt_enc))
device = "cuda:0"

encoding = tokenizer(prmpt, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
print(response)

In [ ]:
prmpt = create_prompt(train_dataset[4])

print(prmpt)

In [ ]:
# prmpt_enc = tokenizer.encode(prmpt)
# model.generate(torch.tensor(prmpt_enc))
device = "cuda:0"

encoding = tokenizer(prmpt, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response

In [ ]:
from peft import LoraModel, LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [18]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_qlora_finetuning",
  #num_train_epochs=5,
  max_steps = 100,
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20,
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

ValueError: Your setup doesn't support bf16/gpu. You need torch>=1.10, using Ampere GPU with cuda>=11.0

In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 3048
    })
})

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=train_dataset,
  eval_dataset=test_dataset
)

In [ ]:
import time
start = time.time()
trainer.train()
print(time.time()- start)